In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
# more info on callbakcs: https://keras.io/callbacks/ model saver is cool too.
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras import optimizers

from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.models import load_model 

from sklearn.model_selection import KFold, StratifiedKFold

import pickle
import time
import numpy as np

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)

In [2]:
pickle_in = open("X.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open("y.pickle","rb")
y = pickle.load(pickle_in)

In [3]:
X = np.array(X)
y = np.array(y)

In [4]:
X.shape

(1640, 224, 224, 3)

In [5]:
y.shape

(1640,)

In [6]:
X = X/255.0

In [7]:
kf = KFold(n_splits = 10, shuffle=True)
                         
skf = StratifiedKFold(n_splits = 10, random_state = 7, shuffle = True)

In [8]:
NAME = "dimentia-simple-CNN-{}".format(int(time.time()))

model = Sequential()

model.add(VGG16(include_top=False, weights='imagenet', input_shape = (224,224,3)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

model.add(Dense(128))
model.add(Dropout(0.2))
model.add(Activation('relu'))
#model.add(BatchNormalization())

model.add(Dense(64))
model.add(Dropout(0.2))
model.add(Activation('relu'))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.summary ()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 25088)             0         
_________________________________________________________________
dense (Dense)                (None, 128)               3211392   
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
activation (Activation)      (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0

In [10]:
# Define per-fold score containers
acc_per_fold = []
loss_per_fold = []

fold_no = 1

for train, val in kf.split(X,y): # skf.split

    tensorboard = TensorBoard(log_dir="/home/trojan/Desktop/dimentia/kfold/logs/{}".format(NAME))

    #es = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=5)
    filepath="model-fold_no{}.h5".format(fold_no)
    #checkpoint = ModelCheckpoint(filepath, monitor='loss', mode='min', verbose=1, save_best_only=True)
    #callbacks_list = checkpoint, es

    #sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

    model.compile(loss='binary_crossentropy',
                  optimizer='sgd',
                  metrics=['accuracy'],
                  )

    
    # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')

    model.fit(X[train], y[train],
              batch_size = 8,
              verbose=1,
              epochs=5,
              callbacks=[tensorboard])
    
    # Generate generalization metrics
    scores = model.evaluate(X[val], y[val], verbose=1)
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])
    
    model.save(filepath)

    # Increase fold number
    fold_no += 1

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/5
  1/185 [..............................] - ETA: 0s - loss: 0.7251 - accuracy: 0.6250WARNING:tensorflow:From /home/trojan/.local/lib/python3.6/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
185/185 [==============================] - 26s 143ms/step - loss: 0.7098 - accuracy: 0.5169
Epoch 2/5
185/185 [==============================] - 26s 142ms/step - loss: 0.6954 - accuracy: 0.5149
Epoch 3/5
185/185 [==============================] - 26s 142ms/step - loss: 0.6948 - accuracy: 0.4905
Epoch 4/5
185/185 [==============================] - 26s 143ms/step - loss: 0.6938 - accuracy: 0.4966
Epoch 5/5
6/6 [==============================] - 1s 85ms/step - loss: 0.6937 - accuracy: 0.4146
Score for fold 1: loss of 0.

185/185 [==============================] - 26s 138ms/step - loss: 0.0016 - accuracy: 1.0000
Epoch 4/5
185/185 [==============================] - 26s 138ms/step - loss: 5.4758e-04 - accuracy: 1.0000
Epoch 5/5
6/6 [==============================] - 1s 84ms/step - loss: 0.0130 - accuracy: 0.9939
Score for fold 8: loss of 0.013038786128163338; accuracy of 99.39024448394775%
------------------------------------------------------------------------
Training for fold 9 ...
Epoch 1/5
185/185 [==============================] - 26s 143ms/step - loss: 0.0012 - accuracy: 0.9993
Epoch 2/5
185/185 [==============================] - 26s 143ms/step - loss: 5.2161e-04 - accuracy: 1.0000
Epoch 3/5
185/185 [==============================] - 27s 144ms/step - loss: 3.0569e-04 - accuracy: 1.0000
Epoch 4/5
185/185 [==============================] - 27s 145ms/step - loss: 2.1171e-04 - accuracy: 1.0000
Epoch 5/5
6/6 [==============================] - 1s 87ms/step - loss: 1.6754e-05 - accuracy: 1.0000
Score for 

In [12]:
# == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')

------------------------------------------------------------------------
Score per fold
------------------------------------------------------------------------
> Fold 1 - Loss: 0.6937023997306824 - Accuracy: 41.46341383457184%
------------------------------------------------------------------------
> Fold 2 - Loss: 0.6916659474372864 - Accuracy: 50.0%
------------------------------------------------------------------------
> Fold 3 - Loss: 0.6907556056976318 - Accuracy: 56.09756112098694%
------------------------------------------------------------------------
> Fold 4 - Loss: 0.7088098526000977 - Accuracy: 60.36585569381714%
------------------------------------------------------------------------
> Fold 5 - Loss: 0.5584264993667603 - Accuracy: 75.0%
------------------------------------------------------------------------
> Fold 6 - Loss: 0.15021176636219025 - Accuracy: 95.12194991111755%
------------------------------------------------------------------------
> Fold 7 - Loss: 0.03313